In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('./data/Occupancy_Estimation.csv')
df.head()

# =========================
# 1. Timestamp + sorting
# =========================
df['timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.sort_values('timestamp').reset_index(drop=True)

# =========================
# 2. Occupancy flag
# =========================
df['is_occupied'] = (df['Room_Occupancy_Count'] > 0).astype(int)

# =========================
# 3. Time difference
# =========================
df['time_diff_min'] = df['timestamp'].diff().dt.total_seconds() / 60

# =========================
# 4. Session break logic
# =========================
# Break session if:
# - occupancy changes
# - OR time gap > 15 min
df['session_break'] = (
    (df['is_occupied'].diff().fillna(0) != 0) |
    (df['time_diff_min'] > 15)
)

# =========================
# 5. Session ID
# =========================
df['session_id'] = df['session_break'].cumsum()

# =========================
# 6. KEEP ONLY OCCUPIED ROWS FOR SESSIONS
# =========================
occupied_df = df[df['is_occupied'] == 1]

# =========================
# 7. Session duration
# =========================
session_duration = (
    occupied_df
    .groupby('session_id')['timestamp']
    .agg(start='min', end='max')
)

session_duration['duration_minutes'] = (
    session_duration['end'] - session_duration['start']
).dt.total_seconds() / 60

# =========================
# 8. Time features
# =========================
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.day_name()

# =========================
# 9. Hourly KPIs
# =========================
hourly_kpi = df.groupby('hour').agg(
    avg_occupancy=('Room_Occupancy_Count', 'mean'),
    max_occupancy=('Room_Occupancy_Count', 'max'),
    utilization_percent=('is_occupied', 'mean')
).reset_index()

hourly_kpi['utilization_percent'] *= 100

# =========================
# 10. Sensor correlation
# =========================
sensor_cols = [
    'S1_Temp','S2_Temp','S3_Temp','S4_Temp',
    'S1_Light','S2_Light','S3_Light','S4_Light',
    'S1_Sound','S2_Sound','S3_Sound','S4_Sound',
    'S5_CO2','S5_CO2_Slope','S6_PIR','S7_PIR'
]

corr = (
    df[sensor_cols + ['Room_Occupancy_Count']]
    .corr()['Room_Occupancy_Count']
    .sort_values(ascending=False)
)

print(corr)

# =========================
# 11. Peak usage hour
# =========================
peak_hour = df.groupby('hour')['is_occupied'].sum().idxmax()
print("Peak usage hour:", peak_hour)




Room_Occupancy_Count    1.000000
S1_Light                0.849058
S3_Light                0.793081
S2_Light                0.788764
S1_Temp                 0.700868
S7_PIR                  0.695138
S2_Temp                 0.671263
S5_CO2                  0.660144
S3_Temp                 0.652047
S6_PIR                  0.633133
S5_CO2_Slope            0.601105
S1_Sound                0.573748
S2_Sound                0.557853
S3_Sound                0.531685
S4_Temp                 0.526509
S4_Sound                0.460287
S4_Light                0.355715
Name: Room_Occupancy_Count, dtype: float64
Peak usage hour: 17


count      6.000000
mean     164.661111
std       78.069605
min       67.400000
25%      119.725000
50%      145.766667
75%      226.183333
max      265.000000
Name: duration_minutes, dtype: float64